In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import json

co_ist=[]
#为减少共现关系中的词语数，只在如下列表中选取字符
useful_words=['China','Deutschland','Europa','USA','Welt','Virus','Russland','EU','Menschen','Land','Pandemie','deutschen','deutsche','chinesischen','Berlin','Politik','Wirtschaft','neue','Frage','Unternehmen','Ukraine','Italien','Länder','BSZ','Schicksalsgemeinschaft','neuen','Deutschlands','Taiwan','Ende','Demokratie','Weber','Corona','Peking','Deutschen','globalen','Xi','Zeit','Krise','Europas','Menschheit','Ländern','Zukunft','Staaten','politischen','Entwicklung','Westen','europäischen','Krieg','chinesische','beiden']

def my_cut(text): 
    # 加载停用词
    stop_words = list(stopwords.words('german')) # 停用词表，去掉不需要的词
    for i in range(len(stop_words)):
        stop_words.append(stop_words[i].capitalize())
    stop_words.extend(('immer','w','http','Chinas','müssen','wurde','schon','heute','viele','Jahren','gerade','Prozent','wäre','dafür','eigenen','weniger','wurden','Milliarden','sehen','kommen','großen','gegenüber','lange','große','brauchen','steht','inzwischen','lässt','Jahre','unserer','vielen','Blick','macht','wirklich','Beispiel','bereits','darauf','wohl','besonders','ersten','fast','geben','tun','hätte','wenig','denen','vergangenen','sollten','damals','darüber','eigene','sogar','klar','lassen','seien','heißt','stellen','kommt','sollen','natürlich','erste','Wochen','Fall','davon','einfach','gehen','stehen','Wer','innen','Woche','gemacht','bleiben','pro','derzeit','täglich','halten','darf','später','deutlich','dabei','braucht','scheint','zurück','daher','sagte','allerdings','helfen','sieht','nie','U','Vorjahr','Jahren','wurde','immer','Prozent','wurden','müssen','Jahre','viele','sagte','drei','gegenüber','heute','Millionen','großen','fünf','schon','Milliarden','bereits','seien','steht','Dienstag','unserer','dafür','Uhr','gerade','weiterhin','insbesondere','wäre','sieht','Mio','kommt','insgesamt','Ve','sogar','große','deutlich ','sagen','kommen','dabei','vielen','vier','aufgrund','bzw','sollen','lange','worden','davon','konnte','Donald','lassen','bleiben','bleibt','derzeit','geworden','ersten','allerdings','sehen','größten','fast','Tag','sollten','wenig','darauf','Angela','heißt','zehn','gilt','letzten','wegen','stehen','jedoch','Dezember','denen','İş','Bankası','Beispiel','liegt','Seite','Woche','macht','eigenen','tun','stellen','nie','a','sowie','jahr','Jahr','t€','T€','gfr','mehr','prozent','uhr','sei','ab','AG','T','gut','mal','seit','sagt','gibt','ganz','wer','ende','teur','rund','jahr','ja','dach','DACH','beim','jahr','geht','gar','zwei','erst','etwa'))

    sents=sent_tokenize(text)
    for sent in sents:
        w_list = []
        #分词、去除停用词
        word_tokens=word_tokenize(sent)
        for w in word_tokens:
            if (w not in stop_words) and w[0].isalpha():
                w_list.append(w)
        co_ist.append(' '.join(w_list))
    #print(co_ist)
    
def str2csv(filePath, s, x):
    '''
    将字符串写入到本地csv文件中
    :param filePath: csv文件路径
    :param s: 待写入字符串(逗号分隔格式)
    '''
    if x=='node':
        with open(filePath, 'w', encoding='utf-8') as f:
            f.write("Label,Weight\r")
            f.write(s)
        print('写入文件成功,请在'+filePath+'中查看')
    else:
        with open(filePath, 'w', encoding='utf-8') as f:
            f.write("Source,Target,Weight\r")
            f.write(s)
        print('写入文件成功,请在'+filePath+'中查看')

def sortDictValue(dict, is_reverse):
    '''
    将字典按照value排序
    :param dict: 待排序的字典
    :param is_reverse: 是否按照倒序排序
    :return s: 符合csv逗号分隔格式的字符串
    '''
    # 对字典的值进行倒序排序,items()将字典的每个键值对转化为一个元组,key输入的是函数,item[1]表示元组的第二个元素,reverse为真表示倒序
    tups = sorted(dict.items(), key=lambda item: item[1], reverse=is_reverse)
    s = ''
    for tup in tups:  # 合并成csv需要的逗号分隔格式
        s = s + tup[0] + ',' + str(tup[1]) + '\n'
    return s

def build_matrix(co_authors_list, is_reverse):
    '''
    根据共同列表,构建共现矩阵(存储到字典中),并将该字典按照权值排序
    :param co_authors_list: 共同列表
    :param is_reverse: 排序是否倒序
    :return node_str: 三元组形式的节点字符串(且符合csv逗号分隔格式)
    :return edge_str: 三元组形式的边字符串(且符合csv逗号分隔格式)
    '''
    node_dict = {}  # 节点字典,包含节点名+节点权值(频数)
    edge_dict = {}  # 边字典,包含起点+目标点+边权值(频数)
    # 第1层循环,遍历整表的每行信息
    for row_authors in co_authors_list:
        row_authors_list = row_authors.split(' ') # 依据','分割每行,存储到列表中
        # 第2层循环
        for index, pre_au in enumerate(row_authors_list): # 使用enumerate()以获取遍历次数index
            if pre_au in useful_words:
                # 若遍历到倒数第一个元素,则无需记录关系,结束循环即可
                if pre_au == row_authors_list[-1]:
                    break
                connect_list = row_authors_list[index+1:]
                # 第3层循环,遍历当前行词后面所有的词,以统计两两词出现的频次
                for next_au in connect_list:
                    A, B = pre_au, next_au
                    #只有当A或B出现频率较高时，才加入边
                    if A in useful_words and B in useful_words:
                        # 统计单个词出现的频次
                        if pre_au not in node_dict:
                            node_dict[pre_au] = 1
                        else:
                            node_dict[pre_au] += 1
                        if next_au not in node_dict:
                            node_dict[next_au] = 1
                        else:
                            node_dict[next_au] += 1

                        # 固定两两词的顺序
                        # 仅计算上半个矩阵
                        if A==B:
                            continue
                        if A > B:
                            A, B = B, A
                        key = A+','+B  # 格式化为逗号分隔A,B形式,作为字典的键
                        # 若该关系不在字典中,则初始化为1,表示词间的共同出现次数
                        if key not in edge_dict:
                            edge_dict[key] = 1
                        else:
                            edge_dict[key] += 1
    #减少edge_dict中键值对，以降低时间复杂度
    new_edge_dict={}
    for key in edge_dict:
        if edge_dict[key] > 5:
            new_edge_dict[key]=edge_dict[key]
    print(len(new_edge_dict))

    # 对得到的字典按照value进行排序
    #with open('./test.txt', 'w', encoding='utf-8') as f:
    #    # 将dic dumps json 格式进行写入
    #    f.write(json.dumps(new_edge_dict))
    node_str = sortDictValue(node_dict, is_reverse)  # 节点
    edge_str = sortDictValue(new_edge_dict, is_reverse)   # 边
    return node_str, edge_str

def main():
    # 从语料库读取疫情后的文本
    with open("myCorpus_after.txt",encoding="utf-8") as f:
        afterText = f.read()

    # 获取数据并存储到列表中
    my_cut(afterText)

    # 根据共同词列表, 构建共现矩阵(存储到字典中), 并将该字典按照权值排序
    node_str, edge_str = build_matrix(co_ist, is_reverse=True)
    #print(edge_str)

    # 将字符串写入到本地csv文件中
    filePath1 = 'node_after.csv'
    filePath2 = 'edge_after.csv'
    str2csv(filePath1,node_str,'node')
    str2csv(filePath2,edge_str,'edge')

main()

94
写入文件成功,请在node_after.csv中查看
写入文件成功,请在edge_after.csv中查看
